# Maxima finding algorithm recreated from implementation in Fiji (ImageJ)

This is a re-implementation of the java plugin written by Michael Schmid and Wayne Rasband for ImageJ. The original java code source can be found in: https://imagej.nih.gov/ij/developer/source/ij/plugin/filter/MaximumFinder.java.html
This code calls a cython compiled version of the code and is much faster than the python only implementation. If compiling the source code is a problem. This can be done by running at the command line. python setup.py build-ext --inplace .

In [1]:
%matplotlib inline
import matplotlib.pylab as plt
import gputools
import numpy as np
import sys
try:
    from find_maxima import find_maxima
except:
    sys.path.append("cython_findmaxima")
    from cython_findmaxima.find_maxima import find_maxima

from PIL import Image
import numpy as np
from scipy import ndimage
import time
import cv2
def find_local_maxima_np(img_data):
    #This is the numpy/scipy version of the above function (find local maxima).
    #Its a bit faster, and more compact code.
    
    #Filter data with maximum filter to find maximum filter response in each neighbourhood
    max_out = ndimage.filters.maximum_filter(img_data,size=3)
    #Find local maxima.
    local_max = np.zeros((img_data.shape))
    local_max[max_out == img_data] = 1
    local_max[img_data == np.min(img_data)] = 0
    return local_max.astype(np.bool)
cntly = np.array([  6,  15,  15,  20,  36,  44,  50,  56,  59,  62,  70,  70,  71,
         76,  83,  89,  93,  96, 109, 111, 126, 133, 134, 135, 136, 148,
        149, 158, 162, 162, 163, 164, 164, 164, 168, 169, 171, 182, 183,
        184, 185, 186, 187, 189, 194, 197, 205, 211, 215, 221, 222, 225,
        226, 228, 231, 233, 241, 243, 244, 245, 245, 248, 252, 257, 263,
        265, 271, 275, 278, 278, 287, 290, 296, 298, 301, 309, 318, 323,
        326, 327, 334, 335, 339, 342, 345, 345, 346, 347, 351, 351, 352,
        358, 366, 366, 371, 372, 375, 379, 383, 384, 396, 411, 414, 415,
        418, 418, 420, 431, 439, 439, 443, 455, 461, 467, 468, 469, 471,
        473, 479, 479])
cntlx = np.array([244, 306, 364, 408, 244, 123, 284, 284,  36, 332, 183, 339, 199,
        362, 160, 226, 125, 446, 320, 102, 159,  92, 127,   0,  17, 429,
        141, 169, 127, 261,  56,  51, 113, 130, 437,  80, 149,  34, 359,
         17,  69, 273,  78,  62, 352, 445, 365, 165,  54,  46,  61, 473,
        458,  52,  43, 361,  86, 407,  51,  43,  65, 455, 168, 187, 230,
        159,  51,  67,  71, 321, 107, 257, 399, 405, 402,  77,  77,  54,
        469, 125, 197,  78,  48, 333,  12,  90, 397, 220,  21, 332, 272,
        160, 361, 443, 363, 253, 363,   4, 482, 106, 379, 329, 131, 338,
        131, 491, 215, 130, 334, 358,  55, 429, 106, 229,  92, 100, 376,
        210, 180, 448])

ModuleNotFoundError: No module named 'is_within'

In [ ]:
img = Image.open('002eggs.png')
ntol = 10 #Noise Tolerance.
img_data = np.array(img).astype(np.float64)

t1 = time.time()


#Should your image be an RGB image.
if img_data.shape.__len__() >2:
    img_data = (np.sum(img_data,2)/3.0)
    
if np.max(img_data) >255 or np.min(img_data)<0:
    print ('warning: your image should be scaled between 0 and 255 (8-bit).')

#Finds the local maxima using maximum filter.
local_max = find_local_maxima_np(img_data)

y,x,out = find_maxima(img_data,local_max.astype(np.uint8),ntol)
print(time.time()-t1)
assert(np.allclose(cntly,y)),"Your values don't match the control"
assert(np.allclose(cntlx,x)),"Your values don't match the control"


In [ ]:
plt.figure(figsize=(24,24))
t1 = time.time()
imgout = (out>=12)
contours,hierarchy = cv2.findContours(np.array(imgout).astype(np.uint8), mode=cv2.RETR_EXTERNAL,method=1)

plt.imshow(imgout)

for cnt in contours[1:]:
    
    
        
        perimeter = float(cv2.arcLength(cnt,True))
        if perimeter >0:
            x,y,w,h = cv2.boundingRect(cnt)

            area = float(cv2.contourArea(cnt))
            circ = 4.*np.pi*(area/(perimeter**2))


            s = float(h)/float(w)
            if s > 1.0:
               pass
            else:
                s = float(h)/float(w)
            #print(solidity)
            if circ >=0.7 and s <1.2:

                plt.plot([x,x+w,x+w,x,x],[y,y,y+h,y+h,y],'g-')
print(time.time()-t1)